In [21]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import neural_network, ensemble, svm, preprocessing, model_selection, metrics
from imblearn import over_sampling

In [22]:
df = pd.read_csv('../assets/data/2019-UT-ASD-Full/2019-UT-ASD-scrapped.csv')

In [23]:
# these are the features that will be used in the modelling part
features = ['RideId', 'TrainId', 'DepartureStation', 'DestinationStation', 
                   'ArrivalTime', 'ArrivalDelay', 'DepartureTime', 'DepartureWindSpeed', 
                   'DepartureVisibility', 'DepartureFog', 'DepartureRain', 'DepartureSnow', 
                   'DepartureThunder', 'DepartureIceFormation', 

                   'DestinationWindSpeed', 'DestinationVisibility', 'DestinationFog', 'DestinationRain',
                   'DestinationSnow', 'DestinationThunder', 'DestinationIceFormation', 'cause_group']

In [24]:
feature_df = df[features]

In [25]:
feature_df['DepartureTime'] = pd.to_datetime(feature_df['DepartureTime']).dt.hour
feature_df['ArrivalTime'] = pd.to_datetime(feature_df['ArrivalTime']).dt.hour

<ipython-input-25-f6988f8061be>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_df['DepartureTime'] = pd.to_datetime(feature_df['DepartureTime']).dt.hour
<ipython-input-25-f6988f8061be>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_df['ArrivalTime'] = pd.to_datetime(feature_df['ArrivalTime']).dt.hour


In [26]:
encoder = preprocessing.LabelEncoder()

In [27]:
feature_df['DepartureStation'] = encoder.fit_transform(feature_df['DepartureStation'])
feature_df['DestinationStation'] = encoder.fit_transform(feature_df['DestinationStation'])

<ipython-input-27-7937afe1ee27>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_df['DepartureStation'] = encoder.fit_transform(feature_df['DepartureStation'])
<ipython-input-27-7937afe1ee27>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_df['DestinationStation'] = encoder.fit_transform(feature_df['DestinationStation'])


In [28]:
# add if a disruption is related to maintenance
feature_df['Maintanence'] = feature_df['cause_group'].apply(lambda cause: True if cause == 'engineering work' else False)

<ipython-input-28-71f58dfe29fd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_df['Maintanence'] = feature_df['cause_group'].apply(lambda cause: True if cause == 'engineering work' else False)


In [29]:
feature_df.drop(columns=['cause_group'], inplace=True)

C:\Users\micha\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [30]:
to_scale = [col for col in feature_df.columns if feature_df[col].max() > 1]

In [31]:
to_scale.remove('ArrivalDelay')

In [32]:
mms = preprocessing.MinMaxScaler()
feature_df[to_scale] = mms.fit_transform(feature_df[to_scale])

<ipython-input-32-26812148682b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_df[to_scale] = mms.fit_transform(feature_df[to_scale])
C:\Users\micha\anaconda3\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [33]:
feature_df.dropna(inplace=True)

<ipython-input-33-2a7612a683b6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_df.dropna(inplace=True)


In [34]:
y = feature_df.pop('ArrivalDelay')
X = feature_df

In [35]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, random_state=42)

In [36]:
svr = svm.SVR()s

In [37]:
svr.fit(X_train, y_train)

SVR()

In [39]:
predictions = svr.predict(X_test)

In [40]:
y_true_pred = pd.DataFrame({
    'y_true': y_test,
    'y_pred': predictions,
})

In [42]:
y_true_pred.describe()

,y_true,y_pred
count,27548.000000,27548.000000
mean,0.956077,0.100735
std,4.759014,0.020290
min,0.000000,-0.075856
25%,0.000000,0.099901
50%,0.000000,0.099987
75%,0.000000,0.100057
max,207.000000,1.399576
